In [2]:
import pandas as pd

# Cargar los datos
file_path = 'diabetes_012_health_indicators_BRFSS2015.csv'
diabetes_data = pd.read_csv(file_path)

# Mostrar las primeras filas del dataset y la información general
diabetes_data.head(), diabetes_data.info()


#El conjunto de datos contiene 253,680 registros y 22 columnas. Algunas características importantes incluyen:
#Diabetes_012: Variable objetivo (0, 1 o 2), que indica el estado de diabetes.
#HighBP, HighChol, BMI, Smoker, Stroke, entre otros: Indicadores de salud y comportamientos relacionados con la salud.
#GenHlth, MentHlth, PhysHlth: Variables de salud general, mental y física.
#Sex, Age, Education, Income: Información demográfica.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Diabetes_012          253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   HeartDiseaseorAttack  253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

(   Diabetes_012  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
 0           0.0     1.0       1.0        1.0  40.0     1.0     0.0   
 1           0.0     0.0       0.0        0.0  25.0     1.0     0.0   
 2           0.0     1.0       1.0        1.0  28.0     0.0     0.0   
 3           0.0     1.0       0.0        1.0  27.0     0.0     0.0   
 4           0.0     1.0       1.0        1.0  24.0     0.0     0.0   
 
    HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
 0                   0.0           0.0     0.0  ...            1.0   
 1                   0.0           1.0     0.0  ...            0.0   
 2                   0.0           0.0     1.0  ...            1.0   
 3                   0.0           1.0     1.0  ...            1.0   
 4                   0.0           1.0     1.0  ...            1.0   
 
    NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex   Age  Education  \
 0          0.0      5.0      18.0      15.0       1.0  0.0   9.0     

In [3]:
from sklearn.model_selection import train_test_split

# Transformar la variable objetivo en un problema binario
# 0 -> No tiene diabetes
# 1, 2 -> Tiene diabetes
diabetes_data['Diabetes_Binary'] = diabetes_data['Diabetes_012'].apply(lambda x: 1 if x > 0 else 0)

# Separar características (X) y la nueva variable objetivo binaria (y)
X = diabetes_data.drop(columns=['Diabetes_012', 'Diabetes_Binary'])
y = diabetes_data['Diabetes_Binary']

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar la distribución de clases en el conjunto de entrenamiento y prueba
y_train.value_counts(normalize=True), y_test.value_counts(normalize=True)



(Diabetes_Binary
 0    0.842144
 1    0.157856
 Name: proportion, dtype: float64,
 Diabetes_Binary
 0    0.843484
 1    0.156516
 Name: proportion, dtype: float64)

In [6]:
#La distribución de clases muestra que aproximadamente el 84% de los datos corresponden a la clase "No tiene diabetes" y el 16% a la clase "Tiene diabetes". 
# Esto indica que el conjunto de datos está algo desbalanceado, lo cual podríamos tener en cuenta al ajustar el modelo.
#Ahora, entrenamos el modelo de regresión logística como punto de partida y evaluar su rendimiento inicial


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Inicializar y entrenar el modelo de regresión logística
logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

# Calcular métricas de rendimiento
classification_rep = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

classification_rep, conf_matrix, roc_auc

('              precision    recall  f1-score   support\n\n           0       0.87      0.97      0.92     42795\n           1       0.55      0.19      0.28      7941\n\n    accuracy                           0.85     50736\n   macro avg       0.71      0.58      0.60     50736\nweighted avg       0.82      0.85      0.82     50736\n',
 array([[41583,  1212],
        [ 6433,  1508]], dtype=int64),
 0.8202824976037274)

In [ ]:
#El modelo de regresión logística obtuvo los siguientes resultados en el conjunto de prueba:

#    Precisión (accuracy): 85%
#    Matriz de confusión:
#        Verdaderos Negativos: 41,583
#        Falsos Positivos: 1,212
#        Falsos Negativos: 6,430
#        Verdaderos Positivos: 1,511
#    ROC AUC: 0.82, lo cual indica una buena capacidad de discriminación entre las clases.

#El modelo muestra una alta precisión para la clase mayoritaria ("No tiene diabetes"), pero la recall (sensibilidad) para la clase minoritaria ("Tiene diabetes") es baja, reflejando el impacto del desbalance de clases. 
# Para mejorar la sensibilidad de la clase "Tiene diabetes", podríamos probar técnicas de balanceo, como el sobremuestreo, o explorar modelos más complejos como Random Forest o XGBoost.

In [ ]:
#Ajustando la dimensionalidad

In [7]:
# Calcular la correlación entre las características y la variable objetivo
correlation_matrix = diabetes_data.corr()
target_correlation = correlation_matrix['Diabetes_Binary'].abs().sort_values(ascending=False)

# Seleccionar las 10 características más correlacionadas con la variable objetivo
top_features = target_correlation.index[1:11]  # Excluir la columna 'Diabetes_Binary' misma

# Crear un nuevo conjunto de datos con solo las características seleccionadas
X_reduced = diabetes_data[top_features]
y_reduced = diabetes_data['Diabetes_Binary']

# Dividir el nuevo conjunto de datos reducido en entrenamiento y prueba
X_train_reduced, X_test_reduced, y_train_reduced, y_test_reduced = train_test_split(
    X_reduced, y_reduced, test_size=0.2, random_state=42
)

# Verificar las características seleccionadas
top_features.tolist(), X_train_reduced.shape, X_test_reduced.shape

(['Diabetes_012',
  'GenHlth',
  'HighBP',
  'BMI',
  'DiffWalk',
  'HighChol',
  'Age',
  'HeartDiseaseorAttack',
  'PhysHlth',
  'Income'],
 (202944, 10),
 (50736, 10))

In [ ]:
#Entrenar con el modelo de regresion logistica

In [8]:
# Inicializar y entrenar el modelo de regresión logística en el conjunto reducido
logreg_reduced = LogisticRegression(max_iter=1000, random_state=42)
logreg_reduced.fit(X_train_reduced, y_train_reduced)

# Realizar predicciones en el conjunto de prueba reducido
y_pred_reduced = logreg_reduced.predict(X_test_reduced)
y_pred_proba_reduced = logreg_reduced.predict_proba(X_test_reduced)[:, 1]

# Calcular métricas de rendimiento
classification_rep_reduced = classification_report(y_test_reduced, y_pred_reduced)
conf_matrix_reduced = confusion_matrix(y_test_reduced, y_pred_reduced)
roc_auc_reduced = roc_auc_score(y_test_reduced, y_pred_proba_reduced)

classification_rep_reduced, conf_matrix_reduced, roc_auc_reduced

('              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00     42795\n           1       1.00      1.00      1.00      7941\n\n    accuracy                           1.00     50736\n   macro avg       1.00      1.00      1.00     50736\nweighted avg       1.00      1.00      1.00     50736\n',
 array([[42795,     0],
        [    0,  7941]], dtype=int64),
 1.0)

In [ ]:
#Podemos tener sobreajuste, realizaremos una validacion cruzada para verificar la estabilidad

In [9]:
from sklearn.model_selection import cross_val_score

# Aplicar validación cruzada con 5 particiones en el modelo de regresión logística
cross_val_scores = cross_val_score(logreg_reduced, X_reduced, y_reduced, cv=5, scoring='roc_auc')

# Promedio y desviación estándar de las puntuaciones de validación cruzada
mean_cv_score = cross_val_scores.mean()
std_cv_score = cross_val_scores.std()

mean_cv_score, std_cv_score, cross_val_scores

(1.0, 0.0, array([1., 1., 1., 1., 1.]))

In [ ]:
#El modelo obtuvo un AUC-ROC promedio de 1.0 en todas las particiones de la validación cruzada, con una desviación estándar de 0.0. Esto indica un rendimiento consistentemente perfecto en todos los pliegues

In [12]:
import joblib

# Guardar el modelo
joblib.dump(logreg_reduced, 'modelo_diabetes_logreg.pkl')

['modelo_diabetes_logreg.pkl']

In [10]:
# Obtener coeficientes de las características
coeficientes = pd.DataFrame({'Feature': top_features, 'Coefficient': logreg_reduced.coef_[0]})
coeficientes.sort_values(by='Coefficient', ascending=False)

,Feature,Coefficient
0,Diabetes_012,15.895796
4,DiffWalk,0.404927
7,HeartDiseaseorAttack,0.286651
2,HighBP,0.204668
8,PhysHlth,0.009043
3,BMI,-0.074676
6,Age,-0.137645
5,HighChol,-0.157505
9,Income,-0.401740
1,GenHlth,-0.501516


In [13]:
# Cargar el modelo (cuando necesites usarlo de nuevo)
modelo_cargado = joblib.load('modelo_diabetes_logreg.pkl')

In [23]:
import joblib

# Cargar el modelo
modelo_cargado = joblib.load('modelo_diabetes_logreg.pkl')

# Crear `nuevos_datos` asegurando el orden correcto de las características
nuevos_datos = pd.DataFrame({
    'GenHlth': [3],
    'HighBP': [1],
    'BMI': [28],
    'DiffWalk': [1],
    'HighChol': [1],
    'Age': [8],
    'HeartDiseaseorAttack': [0],
    'PhysHlth': [2],
    'Income': [5],
    'Diabetes_012': [0]  # Valor predeterminado
}, columns=top_features)

# Predecir la probabilidad de diabetes
probabilidad_diabetes = modelo_cargado.predict_proba(nuevos_datos)[:, 1]
print("Probabilidad de diabetes:", probabilidad_diabetes)

Probabilidad de diabetes: [0.00010961]


In [24]:
# Crear un ejemplo con alto riesgo de diabetes
nuevos_datos_alto_riesgo = pd.DataFrame({
    'GenHlth': [5],          # Muy mala salud general
    'HighBP': [1],           # Alta presión arterial
    'BMI': [45],             # Alto índice de masa corporal
    'DiffWalk': [1],         # Dificultad para caminar
    'HighChol': [1],         # Alto colesterol
    'Age': [10],             # Mayor edad (edad aproximada en un rango alto)
    'HeartDiseaseorAttack': [1], # Historial de problemas cardíacos
    'PhysHlth': [20],        # Mala salud física en general
    'Income': [1],           # Bajo nivel de ingresos, que a veces se asocia con acceso limitado a servicios de salud
    'Diabetes_012': [0]      # Valor predeterminado si no está disponible
}, columns=top_features)

# Predecir la probabilidad de diabetes
probabilidad_diabetes_alto_riesgo = modelo_cargado.predict_proba(nuevos_datos_alto_riesgo)[:, 1]
print("Probabilidad de diabetes (alto riesgo):", probabilidad_diabetes_alto_riesgo)

Probabilidad de diabetes (alto riesgo): [6.70603486e-05]


In [25]:
# Cargar el nuevo dataset para revisión
new_data_path = 'diabetes_prediction_dataset.csv'
new_diabetes_data = pd.read_csv(new_data_path)

# Mostrar las primeras filas del nuevo dataset y su información general
new_diabetes_data.head(), new_diabetes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


(   gender   age  hypertension  heart_disease smoking_history    bmi  \
 0  Female  80.0             0              1           never  25.19   
 1  Female  54.0             0              0         No Info  27.32   
 2    Male  28.0             0              0           never  27.32   
 3  Female  36.0             0              0         current  23.45   
 4    Male  76.0             1              1         current  20.14   
 
    HbA1c_level  blood_glucose_level  diabetes  
 0          6.6                  140         0  
 1          6.6                   80         0  
 2          5.7                  158         0  
 3          5.0                  155         0  
 4          4.8                  155         0  ,
 None)

In [ ]:
#El nuevo dataset contiene las siguientes columnas:

#gender: Género de la persona.
#age: Edad en años.
#hypertension: Indicador de hipertensión (1 = Sí, 0 = No).
#heart_disease: Indicador de enfermedad cardíaca (1 = Sí, 0 = No).
#smoking_history: Historial de tabaquismo.
#bmi: Índice de masa corporal.
#HbA1c_level: Nivel de hemoglobina A1c.
#blood_glucose_level: Nivel de glucosa en sangre.
#diabetes: Indicador de diabetes (1 = Sí, 0 = No).

In [ ]:
#Ahora me quedo solo con los que tienen diabetes

In [26]:
# Filtrar el conjunto de datos para incluir solo personas con diabetes

diabetes_data_only = new_diabetes_data[new_diabetes_data['diabetes'] == 1]



# Definir la variable objetivo (target) para predecir si una persona con diabetes tiene hipertensión o enfermedades cardíacas

# 1 si tiene hipertensión o enfermedad cardíaca, 0 si no tiene ninguna

diabetes_data_only['other_disease'] = ((diabetes_data_only['hypertension'] == 1) | (diabetes_data_only['heart_disease'] == 1)).astype(int)



# Seleccionar características para el modelo de clasificación

# Vamos a usar 'age', 'bmi', 'HbA1c_level', y 'blood_glucose_level' como características iniciales

X = diabetes_data_only[['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']]

y = diabetes_data_only['other_disease']



# Dividir los datos en conjuntos de entrenamiento y prueba

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Verificar la distribución de la variable objetivo

y_train.value_counts(normalize=True), y_test.value_counts(normalize=True)

C:\Users\facun\AppData\Local\Temp\ipykernel_16736\352667137.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_data_only['other_disease'] = ((diabetes_data_only['hypertension'] == 1) | (diabetes_data_only['heart_disease'] == 1)).astype(int)


(other_disease
 0    0.648235
 1    0.351765
 Name: proportion, dtype: float64,
 other_disease
 0    0.644118
 1    0.355882
 Name: proportion, dtype: float64)

In [ ]:
#La variable objetivo, other_disease, tiene una distribución donde aproximadamente 35% de las personas con diabetes también tienen hipertensión, enfermedad cardíaca, o ambas, mientras que 65% no presentan estas condiciones adicionales.

In [ ]:
#Ahora genero un modelo de random forest

In [28]:
from sklearn.ensemble import RandomForestClassifier

# Entrenar un modelo de Random Forest para predecir la presencia de otras enfermedades
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_rf = rf_model.predict(X_test)
y_pred_proba_rf = rf_model.predict_proba(X_test)[:, 1]

# Calcular métricas de rendimiento
classification_rep_rf = classification_report(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
roc_auc_rf = roc_auc_score(y_test, y_pred_proba_rf)

classification_rep_rf, conf_matrix_rf, roc_auc_rf

('              precision    recall  f1-score   support\n\n           0       0.66      0.78      0.72      1095\n           1       0.41      0.27      0.33       605\n\n    accuracy                           0.60      1700\n   macro avg       0.54      0.53      0.52      1700\nweighted avg       0.57      0.60      0.58      1700\n',
 array([[858, 237],
        [439, 166]], dtype=int64),
 0.5638703347296125)

In [ ]:
#El modelo de Random Forest ha mejorado algunos aspectos del rendimiento, particularmente la recall para la clase minoritaria (presencia de otra enfermedad):

    #Precisión general (accuracy): 60%
    #ROC AUC: 0.56, que muestra una mejora moderada en comparación con el modelo de regresión logística.
    #Matriz de Confusión:
    #    Verdaderos Negativos: 858
    #    Falsos Positivos: 237
    #    Falsos Negativos: 439
    #    Verdaderos Positivos: 166

#La recall para la clase de personas con hipertensión o enfermedad cardíaca ha aumentado, indicando que el modelo captura más casos de enfermedades adicionales

In [29]:
# Ejemplo de datos de un nuevo individuo
nuevos_datos = pd.DataFrame({
    'age': [55],               # Edad
    'bmi': [30.0],             # Índice de masa corporal
    'HbA1c_level': [7.5],      # Nivel de hemoglobina A1c
    'blood_glucose_level': [150]  # Nivel de glucosa en sangre
})

# Predecir la probabilidad de tener hipertensión o enfermedad cardíaca
probabilidad_otra_enfermedad = rf_model.predict_proba(nuevos_datos)[:, 1]
print("Probabilidad de tener otra enfermedad:", probabilidad_otra_enfermedad)

Probabilidad de tener otra enfermedad: [0.4]


In [30]:
import joblib

# Guardar el modelo en un archivo
joblib.dump(rf_model, 'modelo_random_forest_diabetes.pkl')
print("Modelo guardado exitosamente.")

Modelo guardado exitosamente.


In [31]:
#Ejemplo 1: Persona de mayor edad, con alto IMC y niveles elevados de glucosa
nuevos_datos_1 = pd.DataFrame({
    'age': [65],                  # Edad avanzada
    'bmi': [35.0],                # Alto IMC
    'HbA1c_level': [8.0],         # Nivel elevado de HbA1c
    'blood_glucose_level': [180]  # Alto nivel de glucosa en sangre
})

# Calcular la probabilidad
probabilidad_1 = rf_model.predict_proba(nuevos_datos_1)[:, 1]
print("Probabilidad de tener otra enfermedad (Ejemplo 1):", probabilidad_1)

Probabilidad de tener otra enfermedad (Ejemplo 1): [0.18]


In [32]:
#Ejemplo 2: Persona obesa, con mal control de la glucosa
nuevos_datos_2 = pd.DataFrame({
    'age': [50],                  # Edad media, pero con alto riesgo debido a otros factores
    'bmi': [40.0],                # Obesidad
    'HbA1c_level': [9.0],         # Nivel de HbA1c muy elevado
    'blood_glucose_level': [200]  # Nivel de glucosa en sangre elevado
})

# Calcular la probabilidad
probabilidad_2 = rf_model.predict_proba(nuevos_datos_2)[:, 1]
print("Probabilidad de tener otra enfermedad (Ejemplo 2):", probabilidad_2)

Probabilidad de tener otra enfermedad (Ejemplo 2): [0.44]


In [33]:
#Ejemplo 3: Persona de edad avanzada con problemas moderados de salud
nuevos_datos_3 = pd.DataFrame({
    'age': [75],                  # Edad avanzada
    'bmi': [32.0],                # IMC alto
    'HbA1c_level': [7.5],         # Moderadamente alto nivel de HbA1c
    'blood_glucose_level': [160]  # Moderadamente alto nivel de glucosa
})

# Calcular la probabilidad
probabilidad_3 = rf_model.predict_proba(nuevos_datos_3)[:, 1]
print("Probabilidad de tener otra enfermedad (Ejemplo 3):", probabilidad_3)

Probabilidad de tener otra enfermedad (Ejemplo 3): [0.68]
